# Bonobo ETL


In [ ]:
# injectable objects
import bonobo
def get_services(**kwargs):
    return {
        'fieldnames': [ 'id', 'city', 'date', 'actual_mean_temp', 'actual_min_temp', 'actual_max_temp', 'actual_precipitation', 'average_precipitation', 'record_precipitation', 'reserved2' ],
        'filename': kwargs['filename']
    }

Once services have been defined, add the functions involved in the process.

In [ ]:
from bonobo.config import use
import csv
csv.register_dialect('pipe_delim', delimiter='|')

from models.weather import Weather

def arg0_to_kwargs(row):
    return bonobo.Bag(**row)

@use('filename')
@use('fieldnames')
def read_recs(filename, fieldnames):
    with open(filename, newline='') as f:
        reader = csv.DictReader(f, dialect='pipe_delim', fieldnames=fieldnames)
        for row in reader:
            print(f'read_recs: row={row}')

            del row[None] # remove unused fields

            weatherrec = Weather(**row)
            print(f'read_recs: weatherrec={weatherrec}')
            
            yield weatherrec

def transform(datain):
    print(f'transform ...')
    return datain

def write_recs(data):
    print(f'write_recs', data)
    pass

Finally, define the graph and run it.

In [ ]:
graph = bonobo.Graph(
    read_recs,
    # transform,
    # arg0_to_kwargs,
    # bonobo.PrettyPrinter())
    write_recs)
bonobo.run(graph, services=get_services(filename='data/inbound/KPHX_with_reserved.dat'))